# ETL Processes
This notebook describes the stages of the ETL process for each the tables before completing the `etl.py` file to load the whole datasets.

In [24]:
#importing libraries
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [25]:
#create a connection to the database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")

#connect to get a cursor that we will use to execute queries
cur = conn.cursor()

In [26]:
#importing the JSON files
def get_files(filepath):
    """ This function uses the filepath of a file in a folder as an input and get a list of JSON files in the folder
    INPUT: filepath of the file
    
    """
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, ETL on the first dataset, `song_data` has been performed, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [27]:
# get the song file
song_files = get_files('data/song_data')

In [28]:
#define the path of the file and print it
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json


In [29]:
#create a Panda df to read the data from JSON file
df_song = pd.read_json(filepath, lines=True)
#show the data in the df_song dataframe
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [30]:
#select columns for song ID, title, artist ID, year, and duration. Convert the array to a list and set it to song_data and select the values from the dataframe
song_data = list(df_song[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])

#see the list
song_data

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [31]:
#insert the data into the songs table
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [32]:
#select columns for artist ID, name, location, latitude, and longitude. Convert the array to a list and set it to artist_data 
#select the values from the dataframe
artist_data = list(df_song[['artist_id','artist_name','artist_location','artist_latitude', 'artist_longitude']].values[0])

#see the values in the list
artist_data

['ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [33]:
#insert the data into the artists table
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [34]:
#get a list of all log JSON files using the get_files function
log_files = get_files('data/log_data')

In [35]:
#define the file path
filepath = log_files[0]

In [36]:
#create a pandas dataframe from the log and show first three rows of the dataframe
df_log= pd.read_json(filepath, lines=True)
df_log.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [37]:
# Filter records by NextSong action from the log and observe if the page column shows the NextSong
df_log = df_log[df_log['page'] == "NextSong"]
df_log.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [38]:
#see the timestamp column values and datatype
df_log.ts.head(3)

0    1541721977796
1    1541722186796
2    1541722347796
Name: ts, dtype: int64

In [39]:
#converting the time into date time format
df_log['start_time'] = pd.to_datetime(df_log['ts'], unit='ms')
df_log['start_time'].head()

0   2018-11-09 00:06:17.796
1   2018-11-09 00:09:46.796
2   2018-11-09 00:12:27.796
3   2018-11-09 00:14:52.796
4   2018-11-09 00:17:44.796
Name: start_time, dtype: datetime64[ns]

In [40]:
#extracting hour, day, week of year, month, year, and weekday from the datetime column 
df_log['hour'] = df_log['start_time'].dt.hour
df_log['day'] = df_log['start_time'].dt.day
df_log['week'] = df_log['start_time'].dt.week
df_log['month'] = df_log['start_time'].dt.month
df_log['year']= df_log['start_time'].dt.year
df_log['weekday']= df_log['start_time'].dt.weekday_name

In [41]:
#see the df_log dataframe to see if the newly extrcted columns are part of the dataframe
df_log.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,start_time,hour,day,week,month,year,weekday
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,...,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,2018-11-09 00:06:17.796,0,9,45,11,2018,Friday


In [42]:
# create a dataframe time_df that contains the timestamp, hour, day, week of year, month, year, and weekday from the ts column and print first 05 rows
time_df = df_log[['start_time','hour','day','week', 'month', 'year', 'weekday']]
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-09 00:06:17.796,0,9,45,11,2018,Friday
1,2018-11-09 00:09:46.796,0,9,45,11,2018,Friday
2,2018-11-09 00:12:27.796,0,9,45,11,2018,Friday
3,2018-11-09 00:14:52.796,0,9,45,11,2018,Friday
4,2018-11-09 00:17:44.796,0,9,45,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table.

In [43]:
#insert the data into the time table
for i, row in time_df.iterrows():
    """
    This procedure insert the information in the time_df dataframe into the time table
    
    INPUT: time_df dataframe
    
    """
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [44]:
#define user_df dataframe
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table.

In [45]:
#insert the data into the users table
for i, row in user_df.iterrows():
    
    """
    This procedure insert the information in the user_df dataframe into the users table
    
    INPUT: user_df dataframe
    
    """
    
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table.

In [46]:
for index, row in df_log.iterrows():
    
    """
    This procedure processes a song file whose filepath has been provided as an arugment.
    It extracts the song information in order to store it into the songs table.
    Then it extracts the artist information in order to store it into the artists table.

    INPUTS: 
    * cur the cursor variable
    * filepath the file path to the song file
    """

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    song_id, artist_id = results if results else None, None
    
    #insert songplay record
    songplay_data = (song_id, row.start_time, row.userId, row.level, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [47]:
conn.close()

# Implement `etl.py`
`etl.py` notebook to implemented based on what is completed in this file.